# Анализ характеристик и стоимости квартир.
Источник данных: выгрузка из базы данных property.db.

*Данные: [Список квартир с размерами помещений](https://github.com/Valery2120/Flat/blob/main/jupyter/test.csv).*


***

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sqlite3

## 1. Загрузка и анализ данных.

In [2]:
con = sqlite3.connect('property.db')

flats = pd.read_sql("SELECT f.flat_id, r.region, c.city, s.street, f.house_number, "
                 "f.flat_number, rm.name, rm.length, rm.width, rm.height FROM room rm "
                 "JOIN flat f ON rm.flat_id = f.flat_id "
                 "JOIN street s ON f.street_id = s.street_id "
                 "JOIN city c ON s.city_id = c.city_id "
                 "JOIN region r ON c.region_id = r.region_id "
                 "ORDER BY city, street, house_number, flat_number", con)
flats

,flat_id,region,city,street,house_number,flat_number,name,length,width,height
0,285,Brest,Baranavichy,Chkalov,5,46,Kitchen,3.69,4.92,2.5
1,285,Brest,Baranavichy,Chkalov,5,46,Bathroom,1.90,3.09,2.5
2,285,Brest,Baranavichy,Chkalov,5,46,Bedroom,3.27,5.76,2.5
3,24,Brest,Baranavichy,Khmelevskogo,39,18,Kitchen,1.55,3.78,2.5
4,24,Brest,Baranavichy,Khmelevskogo,39,18,Bathroom,1.52,2.64,2.5
...,...,...,...,...,...,...,...,...,...,...
1269,180,Minsk,Zhodino,Rumyantseva,74,2,Kitchen,2.54,2.12,2.5
1270,180,Minsk,Zhodino,Rumyantseva,74,2,Bathroom,1.94,1.81,2.5
1271,180,Minsk,Zhodino,Rumyantseva,74,2,Bedroom,3.52,5.90,2.5
1272,180,Minsk,Zhodino,Rumyantseva,74,2,Bedroom,3.54,3.97,2.5


In [3]:
flats['square'] = (flats['width'] * flats['length']).round(decimals = 2)
flats['cost'] = flats['square'] * 600
flats['volume'] = (flats['square'] * flats['height']).round(decimals = 2)

flats['address'] = flats['city'] + ', ' + flats['street'] + ' ' + flats['house_number'].apply(str) + '/' + flats['flat_number'].apply(str)

Определяем количество комнат в квартирах и группируем исходные данные по адресам.

In [4]:
flats.loc[flats['name'] == 'Bedroom', 'rooms'] = 1

In [5]:
flats_info = flats[['address', 'rooms', 'square', 'volume', 'cost']].groupby(['address']).sum()

##### Сводные данные с характеристиками по каждой квартире  

In [6]:
flats_info = flats_info.reset_index()
flats_info

,address,rooms,square,volume,cost
0,"Baranavichy, Chkalov 5/46",1.0,42.86,107.16,25716.0
1,"Baranavichy, Khmelevskogo 39/18",2.0,42.48,106.19,25488.0
2,"Baranavichy, Malinina 58/57",1.0,34.59,86.48,20754.0
3,"Baranavichy, Shchorsa 52/66",2.0,52.25,130.63,31350.0
4,"Bobruisk, Kozlova 51/5",1.0,58.42,146.04,35052.0
...,...,...,...,...,...
332,"Zhodino, Lomonosov 28/100",1.0,46.47,125.47,27882.0
333,"Zhodino, Novaya 12/13",1.0,23.66,59.15,14196.0
334,"Zhodino, Novaya 53/81",1.0,29.58,73.95,17748.0
335,"Zhodino, Paris 43/5",1.0,41.66,104.14,24996.0


##### Определение квартир с наибольшей площадью, объемом и стоимостью

In [7]:
flats_info.sort_values(by='square', ascending=False).head(10)

,address,rooms,square,volume,cost
272,"Slonim, Zaporozhskaya 8/41",3.0,115.29,288.22,69174.0
126,"Lida, Koshevogo 54/45",3.0,107.60,269.01,64560.0
122,"Lepel, Zaslonova 38/7",3.0,102.77,256.92,61662.0
80,"Klimovichi, Parovoznyy 49/70",3.0,101.98,254.96,61188.0
288,"Soligorsk, Stebeneva 16/57",3.0,100.80,262.09,60480.0
73,"Ivatsevichi, Kizhevatova 48/56",3.0,100.12,250.29,60072.0
120,"Lepel, Sechenova 9/47",3.0,99.52,248.81,59712.0
173,"Mozyr, Semenova 72/60",4.0,99.17,247.94,59502.0
189,"Novopolotsk, Altayskiy 39/40",3.0,98.37,245.93,59022.0
300,"Vitebsk, Fomina 52/18",3.0,98.18,245.46,58908.0


In [8]:
flats_info.sort_values(by='volume', ascending=False).head(10)

,address,rooms,square,volume,cost
272,"Slonim, Zaporozhskaya 8/41",3.0,115.29,288.22,69174.0
126,"Lida, Koshevogo 54/45",3.0,107.60,269.01,64560.0
153,"Minsk, Petrovshchina 32/85",3.0,88.57,265.71,53142.0
288,"Soligorsk, Stebeneva 16/57",3.0,100.80,262.09,60480.0
122,"Lepel, Zaslonova 38/7",3.0,102.77,256.92,61662.0
80,"Klimovichi, Parovoznyy 49/70",3.0,101.98,254.96,61188.0
73,"Ivatsevichi, Kizhevatova 48/56",3.0,100.12,250.29,60072.0
120,"Lepel, Sechenova 9/47",3.0,99.52,248.81,59712.0
173,"Mozyr, Semenova 72/60",4.0,99.17,247.94,59502.0
189,"Novopolotsk, Altayskiy 39/40",3.0,98.37,245.93,59022.0


In [9]:
flats_info.sort_values(by='cost', ascending=False).head(10)

,address,rooms,square,volume,cost
272,"Slonim, Zaporozhskaya 8/41",3.0,115.29,288.22,69174.0
126,"Lida, Koshevogo 54/45",3.0,107.60,269.01,64560.0
122,"Lepel, Zaslonova 38/7",3.0,102.77,256.92,61662.0
80,"Klimovichi, Parovoznyy 49/70",3.0,101.98,254.96,61188.0
288,"Soligorsk, Stebeneva 16/57",3.0,100.80,262.09,60480.0
73,"Ivatsevichi, Kizhevatova 48/56",3.0,100.12,250.29,60072.0
120,"Lepel, Sechenova 9/47",3.0,99.52,248.81,59712.0
173,"Mozyr, Semenova 72/60",4.0,99.17,247.94,59502.0
189,"Novopolotsk, Altayskiy 39/40",3.0,98.37,245.93,59022.0
300,"Vitebsk, Fomina 52/18",3.0,98.18,245.46,58908.0


## 2. Результирующие данные и выводы.

In [10]:
categories = ['Количество квартир:',
                  'Количество 1-к. кв.:',
                  'Количество 2-к. кв.:',
                  'Количество 3-к. кв.:',
                  'Квартира с комнатой наименьшей площади:',
                  'Квартира с комнатой наибольшей площади:',
                  'Квартира с ванной наименьшего объема:',
                  'Квартира с ванной наибольшего объема:']

values = [len(flats_info),
            len(flats_info['address'][flats_info['rooms'] == 1]),
            len(flats_info['address'][flats_info['rooms'] == 2]),
            len(flats_info['address'][flats_info['rooms'] == 3]),
            flats['address'].iloc[flats['square'][flats['name'] == 'Bedroom'].idxmin()],
            flats['address'].iloc[flats['square'][flats['name'] == 'Bedroom'].idxmax()],
            flats['address'].iloc[flats['volume'][flats['name'] == 'Bathroom'].idxmin()],
            flats['address'].iloc[flats['volume'][flats['name'] == 'Bathroom'].idxmax()]]

In [11]:
flats_analysis = pd.DataFrame({'Категории': categories, 'Значения': values})
flats_analysis

,Категории,Значения
0,Количество квартир:,337
1,Количество 1-к. кв.:,154
2,Количество 2-к. кв.:,96
3,Количество 3-к. кв.:,82
4,Квартира с комнатой наименьшей площади:,"Brest, Orsha 19/63"
5,Квартира с комнатой наибольшей площади:,"Brest, Radishchev 43/28"
6,Квартира с ванной наименьшего объема:,"Kalinkovichi, Tukhachevskogo 60/97"
7,Квартира с ванной наибольшего объема:,"Smorgon, Bersona 4/58"


### Работа по анализу характеристик квартир состояла из следующих этапов:

1. Загрузка данных.
2. Определение общего количества квартир в списке и количества 1, 2, 3-комнатных квартир.
3. Вычисление площадей и объемов квартир.
4. Определение квартиры с наименьшей и наибольшей комнатой по площади.
5. Определение квартиры с наименьшей и наибольшей ванной по объему.
6. Вычисление стоимости квартир исходя из 600 $ за 1 м2.


### Выводы.
Всего в выборке было представлено 337 квартир: 82 - трехкомнатная, 96 - двухкомнатные, 154 - однакомнатные.
Так как стоимость 1 кв. метра одинакова для всех квартир соответственно квартира с наибольшей площадью является самой дорогой по стоимости (Slonim, Zaporozhskaya 8/41). При этом в десятку квартир с наибольшей площадью, объемом и стоимостью вошли 2 квартиры из Lepel. 